## Reference

[https://python.langchain.com/docs/integrations/chat/ollama/](https://python.langchain.com/docs/integrations/chat/ollama/)

In [1]:
import json

from langchain_community.chat_models import ChatOllama

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
    
def get_chat_history(eval_data):
    str_chat_history = []
    for msg in eval_data['msg']:
        str_chat_history.append(f"{msg['role']} : {msg['content']}")

    return '\n'.join(str_chat_history)

In [3]:
eval_file = "../dataset/eval.jsonl"
eval_dataset = load_jsonl(eval_file)

In [4]:
model = ChatOllama(model="eeve-10.8b-q8:latest")

In [5]:
prompt_context1 = (
    "주어진 '대화내용'을 정리해서 '하나의 질문을 생성'해주세요.\n"

    "예제 입력\n"
    "'user : 기억 상실증 걸리면 너무 무섭겠다.\n' 'assistant : 네 맞습니다.\n' 'user : 어떤 원인 때문에 발생하는지 궁금해.\n'"
    "예제 출력\n"
    "'기억 상실증에 걸리는 원인은 무엇인가요?'"

    "대화내용 : {chat_history}\n\n"

    "답변:"
)

prompt1 = ChatPromptTemplate.from_template(prompt_context1)
chain1 = prompt1 | model | StrOutputParser()

In [6]:
prompt_context2 = (
    # "주어진 질문이 '상식'을 물어보는 것인지 판단하세요.\n"
    "주어진 질문이 일반적인 지식이나 사실에 대한 질문인지 판단하세요.\n"

    "만약 개인적인 감정, 의견, 일상 대화처럼 질문이 상식을 묻는 질문이 아닌 경우 False를, 상식을 묻는 질문이 맞다면 True를 반환하세요.\n"
    "당신의 답변은 반드시 True 또는 False로만 구성되어야 하며, 다른 문장은 생성하지 않아야 합니다.\n"

    "예제 입력1\n"
    "평형분극비율이 뭐야?\n"
    "예제 출력1\n"
    "True\n"

    "예제 입력2\n"
    "다음 세대의 꽃 색깔은 무엇인가요?\n"
    "예제 출력2\n"
    "True\n"

    "예제 입력3\n"
    "요새 너무 힘드네..\n"
    "예제 출력3\n"
    "False\n"

    "예제 입력4\n"
    "너 모르는 것도 있어?\n"
    "예제 출력4\n"
    "False\n"

    "예제 입력5\n"
    "니가 대답을 잘해줘서 너무 신나!\n"
    "예제 출력5\n"
    "False\n"


    "예제 입력6\n"
    "우울한데 신나는 얘기 좀 해줘!\n"
    "예제 출력6\n"
    "False\n"

    "예제 입력7\n"
    "너는 누구야?\n"
    "예제 출력7\n"
    "False\n"

    "예제 입력8\n"
    "너 잘하는게 뭐야?\n"
    "예제 출력8\n"
    "False\n"

    "질문 : {query}\n\n"

    "답변:"
)

prompt2 = ChatPromptTemplate.from_template(prompt_context2)
chain2 = prompt2 | model | StrOutputParser()

In [7]:
data1 = eval_dataset[0]
print(data1['msg'])

sample1 = get_chat_history(data1)
print(sample1)

[{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
user : 나무의 분류에 대해 조사해 보기 위한 방법은?


In [8]:
data2 = eval_dataset[8]
print(data2['msg'])

sample2 = get_chat_history(data2)
print(sample2)

[{'role': 'user', 'content': '이란 콘트라 사건이 뭐야'}, {'role': 'assistant', 'content': '이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.'}, {'role': 'user', 'content': '이 사건이 미국 정치에 미친 영향은?'}]
user : 이란 콘트라 사건이 뭐야
assistant : 이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.
user : 이 사건이 미국 정치에 미친 영향은?


In [9]:
gen_result1 = chain1.invoke({"chat_history" : sample1})

print()
print(gen_result1)

나무의 분류를 연구하기 위해 사용할 수 있는 방법에는 어떤 것들이 있나요?
나무의 분류를 연구하기 위해 사용할 수 있는 방법에는 어떤 것들이 있나요?


In [10]:
gen_result2 = chain1.invoke({"chat_history" : sample2})

print()
print(gen_result2)

이란-콘트라 사건이 미국 정치에 어떤 영향을 미쳤습니까?
이란-콘트라 사건이 미국 정치에 어떤 영향을 미쳤습니까?


In [11]:
# 초기화
num_correct = 0  # 모델이 정답으로 판단한 횟수
total = 0        # 전체 데이터 수
false_count = 0  # 모델이 False로 판단한 횟수
incorrect_examples = []  # 모델이 틀린 예시를 저장할 리스트

out_query_list = [276, 261, 283, 32, 94, 90, 220, 245, 229, 247, 67, 57, 2, 227, 301, 222, 83, 64, 103, 218]

for idx, eval_data in enumerate(eval_dataset):
    eval_id = eval_data['eval_id']
    chat_len = len(eval_data['msg'])
    chats = get_chat_history(eval_data)
    
    if chat_len > 1:
        query = chain1.invoke({"chat_history": chats})
        _ = query
    else:
        query = chats.split(':')[1].strip()
    
    check = chain2.invoke({"query": query})
    print(f"eval_id: {eval_id}, query: {query}, model_output: {check}\n")
    
    # 기대되는 정답 설정
    expected_check = 'False' if eval_id in out_query_list else 'True'
    
    # 모델의 출력과 기대되는 정답 비교
    if check == expected_check:
        num_correct += 1
    else:
        # 틀린 예시 저장
        incorrect_examples.append({
            'eval_id': eval_id,
            'query': query,
            'expected': expected_check,
            'model_output': check
        })
    
    # 모델이 False로 판단한 경우 카운트 증가
    if check == 'False':
        false_count += 1
    
    total += 1  # 총 데이터 수 증가

# 정답률 계산 및 출력
accuracy = (num_correct / total) * 100
print(f"모델의 정답률: {accuracy:.2f}%")
print(f"모델이 False로 판단한 횟수: {false_count}회\n")

# 모델이 틀린 예시 출력
if incorrect_examples:
    print("모델이 틀린 예시:")
    for example in incorrect_examples:
        print(f"eval_id: {example['eval_id']}, query: {example['query']}")
        print(f"예상 답변: {example['expected']}, 모델의 답변: {example['model_output']}\n")
else:
    print("모델이 모든 예시를 맞췄습니다.")

Trueeval_id: 78, query: 나무의 분류에 대해 조사해 보기 위한 방법은?, model_output: True

Trueeval_id: 213, query: 각 나라에서의 공교육 지출 현황에 대해 알려줘., model_output: True

기억 상실증의 가능한 원인에 대해 설명해 주실 수 있나요?Trueeval_id: 107, query: 기억 상실증의 가능한 원인에 대해 설명해 주실 수 있나요?, model_output: True

Trueeval_id: 81, query: 통학 버스의 가치에 대해 말해줘., model_output: True

Trueeval_id: 280, query: Dmitri Ivanovsky가 누구야?, model_output: True

Trueeval_id: 10, query: 피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?, model_output: True

Trueeval_id: 100, query: 헬륨이 다른 원소들과 반응을 잘 안하는 이유는?, model_output: True

Trueeval_id: 279, query: 문맹 비율이 사회 발전에 미치는 영향은?, model_output: True

이란-콘트라 사건이 미국 정치에 어떤 영향을 미쳤습니까?Trueeval_id: 42, query: 이란-콘트라 사건이 미국 정치에 어떤 영향을 미쳤습니까?, model_output: True

Trueeval_id: 308, query: 자기장이 얼마나 센지 표현하는 방식은?, model_output: True

Trueeval_id: 205, query: 피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?, model_output: True

Trueeval_id: 289, query: 글리코겐의 분해는 인체에서 왜 필요한가?, model_output: True

Trueeval_id: 268, query: 빗방울이 점점 커지게 되는 요인은?, model_output: True



: 